In [1]:
import os
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet
from gensim.models import Word2Vec, FastText
import pickle

In [2]:
# df_train = pd.read_json(path_or_buf='../data//gazeta_train.jsonl', lines=True)
# df_test = pd.read_json(path_or_buf='../data/gazeta_test.jsonl', lines=True)
# df_val = pd.read_json(path_or_buf='../data/gazeta_val.jsonl', lines=True)

In [3]:
stopworlds = nltk.corpus.stopwords.words('russian')
stemmer = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
v_word_tokenize = np.vectorize(nltk.tokenize.word_tokenize)

In [4]:
def preprocess_txt(line):
    return list({
        lemmatizer.lemmatize(
            stemmer.stem(w), wordnet.VERB
        ) for w in nltk.tokenize.word_tokenize(line) if w not in stopworlds
    })

In [124]:
with pd.read_json(path_or_buf='../data//gazeta_train.jsonl', lines=True, chunksize=100) as reader:
    if os.path.isfile('../data//gazeta_train.csv'):
        os.remove('../data//gazeta_train.csv')
    for chunk in reader:
        chunk['text_mod'] = chunk['text'].apply(
            lambda x: preprocess_txt(x)
        )
        chunk.to_csv('../data//gazeta_train.csv', mode='a', index=False, sep='&')


In [ ]:
df_train = pd.read_csv('../data//gazeta_train.csv', sep='&')

In [ ]:
df_train.columns

In [8]:
modelW2V = Word2Vec(sentences=df_train['text_mod'], vector_size=300, min_count=1, window=5, workers=6)

In [9]:
with open('modelW2V.pickle', 'wb') as f:
    pickle.dump(modelW2V, f)

In [7]:
modelFT = FastText(sentences=df_train['text_mod'], vector_size=300, min_count=1, window=5, workers=6)

In [8]:
with open('modelFT.pickle', 'wb') as f:
    pickle.dump(modelFT, f)

In [5]:
with open('modelW2V.pickle', 'rb') as f:
    modelW2V = pickle.load(f)

In [6]:
with open('modelFT.pickle', 'rb') as f:
    modelFT = pickle.load(f)

In [9]:
def get_response(model, question, topn = 5):
    df = pd.DataFrame(columns=['p', 'text'])
    with pd.read_csv('../data//gazeta_train.csv', sep='&', chunksize=1000) as rider:
        for chunk in rider:
            chunk['p'] = model.wv.wmdistance(question, chunk['text'])
            chunk_top = chunk.nsmallest(n=topn, columns=['p'], keep='first')
            df = pd.concat([df, chunk_top])
    df_top = df.nsmallest(n=topn, columns=['p'], keep='first')
    return df_top

In [11]:
get_response(modelFT, 'Когда отменят транспортный налог', topn=5)

,p,text,date,url,summary,title,text_mod
50000,1.321085,В «день тишины» накануне выборов президента Ук...,2019-03-21 12:01:52,https://www.gazeta.ru/culture/2019/03/21/a_122...,Накануне выборов президента Украины телеканал ...,"День тишины? Нет, не слышали: как Зеленский об...","['подтверждения', 'символом', 'зеленский', 'пр..."
50001,1.321085,«Я попросила президента о досрочной отставке. ...,2019-03-21 12:03:29,https://www.gazeta.ru/politics/2019/03/21_a_12...,Губернатор Мурманской области Марина Ковтун по...,«Конфликтный стиль»: почему Ковтун ушла в отст...,"['выборах', 'сообществе', 'влиянию', 'интересы..."
50002,1.321085,Бывший лечащий врач Юлии Началовой Василий Шур...,2019-03-21 12:03:52,https://www.gazeta.ru/social/2019/03/21/122559...,Толчком для карьеры Началовой послужило задерж...,«Задержание Началовой ДПС стало новым толчком ...,"['снята', 'заражения', 'подошли', 'окружили', ..."
50003,1.321085,21 марта в японской Сайтаме был разыгран первы...,2019-03-21 12:14:29,https://www.gazeta.ru/sport/2019/03/21/a_12256...,Две российские пары завоевали медали на чемпио...,«Китайцы непобедимы»: почему российские пары п...,"['десять', 'появиться', 'первую', 'траньков', ..."
50004,1.321085,Экс-мэр Москвы Юрий Лужков считает оправданной...,2019-03-21 12:18:15,https://www.gazeta.ru/politics/2019/03/21_a_12...,Бывший мэр Москвы Юрий Лужков поддержал инициа...,«По пьяни ельцинской»: что Лужков требует от У...,"['председателя', 'истории', 'первую', 'россией..."


In [12]:
get_response(modelW2V, 'Когда отменят транспортный налог', topn=5)

,p,text,date,url,summary,title,text_mod
0,inf,Сегодня транспортный налог начисляется в завис...,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,"['малолитражек', 'доходов', 'бензин', '6000', ..."
1,inf,Словосочетание «музыкальный кинофестиваль» уже...,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал","['пор', 'балеарике', 'наш', 'андеграундной', '..."
2,inf,После более чем 12-часовых консультаций Совет ...,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,"['израильский', 'председателя', 'человек', 'от..."
3,inf,"Жертвами урагана «Агата», обрушившегося на Цен...",2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,"['пешком', 'доходов', 'центральную', 'человек'..."
4,inf,Решение ограничить рост тарифов естественных м...,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,"['попытаюся', 'ряд', 'андрей', 'те', 'а', 'ген..."
